In [1]:
import pandas as pd
import numpy as np
from sklearn import tree

In [2]:
animal_df = pd.read_csv('animals-training.csv')
animal_df.head()

,Legs,Body Covering,Animal
0,0,scales,snake
1,0,scales,snake
2,2,feathers,bird
3,2,feathers,bird
4,2,feathers,bird


In [3]:
col_list = list(animal_df.columns)
objList = []
for col in col_list:
    if animal_df.dtypes[col] == object:
        objList.append(col)
objList

['Body Covering', 'Animal']

In [4]:
df_cats = pd.get_dummies(animal_df[objList[:-1]], prefix_sep='_', drop_first=False)
df_cats.head()

,Body Covering_feathers,Body Covering_furry,Body Covering_hide,Body Covering_scales
0,0,0,0,1
1,0,0,0,1
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0


In [5]:
numList = [item for item in col_list if item not in objList]
numList
df_nums = animal_df[numList]

In [6]:
df_vars = pd.concat([df_nums, df_cats],axis=1)
X = df_vars
features = list( df_vars.columns)
y = animal_df.iloc[:, -1].values 

In [7]:
clf=tree.DecisionTreeClassifier(criterion='gini',splitter='best')
clf=clf.fit(X,y)
tree1 = clf.tree_

In [8]:
clf2=tree.DecisionTreeClassifier(criterion='entropy',splitter='best')
clf2=clf.fit(X,y)
tree2 = clf2.tree_

In [9]:
def optimal_split(df, tree_):
    #get initial Gini/Entropy
    count = df['Animal'].value_counts().reset_index().rename(columns={"index": "Animal", 'Animal': "count"})
    count['p(x)'] = np.round(count['count']/sum(count['count']),2)
    count['p(1-x)'] = np.subtract(1, count['p(x)'])
    count['log(px)'] = np.log2(count['p(x)'])
    count['entropy'] = np.round(count['p(x)'] * -count['log(px)'],2)
    count['gini'] = np.round(count['p(x)']*count['p(1-x)'],2)
    initial_gini = np.round(np.sum(count['gini']),2)
    initial_entropy = np.round(np.sum(count['entropy']),2)
    gini_values = tree_.impurity[1:3]
    tree_extract = tree.export_text(clf, feature_names=features)
    split_string = tree_extract.split('|')
    first_feature = split_string[1].split('---') 
    optimal_split = first_feature[1]
    return optimal_split, initial_entropy, initial_gini, gini_values

In [10]:
#gini tree
os1, ig1, ie1, gv1 = optimal_split(animal_df, tree1)
print(f'The initial ginie value was {ig1}')
print(f'The initial entropy value was {ie1}')
print(f'The gini values after the first split were {gv1}')
print(f'The optimal split is on {os1}')

#entropy tree
os2, ig2, ie2, gv2 = optimal_split(animal_df, tree2)
print(f'The initial ginie value was {ig2}')
print(f'The initial entropy value was {ie2}')
print(f'The gini values after the first split were {gv2}')
print(f'The optimal split is on {os1}')

The initial ginie value was 2.56
The initial entropy value was 0.81
The gini values after the first split were [0.7768595 0.71875  ]
The optimal split is on  Body Covering_scales <= 0.50

The initial ginie value was 2.56
The initial entropy value was 0.81
The gini values after the first split were [0.7768595 0.71875  ]
The optimal split is on  Body Covering_scales <= 0.50

